# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [4]:
%nuclio config spec.build.baseImage = "mlrun/ml-models:0.4.6"

%nuclio: setting spec.build.baseImage to 'mlrun/ml-models:0.4.6'


In [5]:
%%nuclio cmd -c
python -m pip install -U kfserving 
python -m pip install numpy cloudpickle v3io

In [6]:
import os
from cloudpickle import load
import kfserving
import numpy as np
from typing import List
from datetime import datetime

In [7]:
import json
import socket
import v3io.dataplane

v3io_client = v3io.dataplane.Client()
hostname = None

def split_path(mntpath=''):
    if mntpath[0] == '/':
        mntpath = mntpath[1:]
    paths = mntpath.split('/')
    container = paths[0]
    subpath = ''
    if len(paths) > 1:
        subpath = mntpath[len(container):]
    return container, subpath
 

def init_stream(stream_path, shards=1):
    global hostname
    hostname = socket.gethostname()
    container, stream_path = split_path(stream_path)
    response = v3io_client.create_stream(container=container,
                                         path=stream_path, 
                                         shard_count=shards,
                                         raise_for_status=v3io.dataplane.RaiseForStatus.never)
    response.raise_for_status([409, 204])
    print(response.__dict__)
    
def push_to_stream(stream_path, data):
    records = [{'data': json.dumps(rec)} for rec in data]
    container, stream_path = split_path(stream_path)
    response = v3io_client.put_records(container=container,
                                       path=stream_path, 
                                       records=records)
    
    #for response_record in response.output.records:
    #    print(response_record.__dict__)

In [8]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, classifier = None):
        super().__init__(name)
        self.model_dir = model_dir
        self.log_stream = os.environ.get('INFERENCE_STREAM', '')
        if self.log_stream:
            init_stream(self.log_stream)
        if classifier:
            self.classifier = classifier
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        if self.model_dir.endswith('.pkl'):
            model_file = self.model_dir
        else:
            for file in os.path.listdir(self.model_dir):
                if file.endswith('.pkl'):
                    model_file = os.pth.join(self.model_dir, file)
                    break
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        start = datetime.now()
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        if self.log_stream:
            data = {'request': body, 'resp': resp, 
                    'class': 'ClassifierModel', 
                    'model': self.name,
                    'host': hostname,
                    'when': str(start), 
                    'microsec': (datetime.now()-start).microseconds}
            push_to_stream(self.log_stream, [data])

        return resp

In [9]:
# nuclio: end-code

# Save

In [10]:
from mlrun import new_model_server
fn = new_model_server('sklearn-server', model_class='ClassifierModel')
fn.spec.description = "generic sklearn model server"
fn.metadata.categories = ['serving', 'models']
fn.metadata.labels = {'author': 'yaronh'}
#print(fn.to_yaml())
fn.export()

[mlrun] 2020-04-25 23:05:39,534 function spec saved to path: function.yaml


# Test

In [13]:
from mlrun import mount_v3io, mlconf
fn.apply(mount_v3io())
fn.set_envs({'SERVING_MODEL_iris_dataset_v1': mlconf.artifact_path + '/models/model.pkl',
             'INFERENCE_STREAM': 'users/admin/tststream'})
fn.deploy(project='sk-project')

[mlrun] 2020-04-25 23:06:45,217 deploy started
[nuclio] 2020-04-25 23:08:47,410 (error) Failed to deploy. Details:
Failed to create/update function


DeployError: cannot deploy 